# 🏋️‍♂️ Tydzień 7, Dzień 2: Lokalny Fine-Tuning (QLoRA)

Dzisiaj przeprowadzimy trening modelu na własnym sprzęcie GPU.
Wykorzystamy technikę **QLoRA** (Quantized Low-Rank Adaptation).

**Co to znaczy?**
1.  **Quantized (4-bit):** Ściskamy model 4-krotnie, żeby zmieścił się w pamięci karty graficznej (VRAM).
2.  **LoRA:** Nie trenujemy całego modelu, ale tylko małe "nakładki" (adaptery).

**Stos technologiczny:**
*   **Model:** `Qwen/Qwen2.5-1.5B-Instruct` (Nowoczesny, lekki model, idealny na start).
*   **Biblioteki:** `bitsandbytes` (kompresja), `peft` (adaptery), `trl` (trening).

---
### 🛠️ 1. Instalacja (Wersja GPU)

Najpierw instalujemy PyTorch z obsługą CUDA a potem resztę narzędzi standardowo.

In [ ]:
# 1. Instalacja PyTorch (Specjalna wersja CUDA 12.8)
# Używamy 'uv pip install', aby wskazać niestandardowy index-url
!uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

# 2. Reszta bibliotek (Standardowe repozytorium)
# Używamy 'uv add', aby dodać je do projektu
!uv add transformers peft bitsandbytes trl accelerate datasets pandas

### 🖥️ 2. Sprawdzenie GPU

Upewnijmy się, że Python widzi Twoją kartę graficzną.

In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    # Sprawdzenie ile pamięci VRAM ma karta
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"VRAM: {vram:.2f} GB")
else:
    raise RuntimeError("❌ Nie wykryto GPU! Sprawdź sterowniki CUDA.")

PyTorch version: 2.9.1+cu128
CUDA available: True
GPU: NVIDIA GeForce RTX 4060
VRAM: 8.59 GB


### 📥 3. Ładowanie Danych

Wczytamy plik `jsonl`, który przygotowaliśmy w Dniu 1.
Upewnij się, że plik `financial_sentiment_train.jsonl` istnieje w folderze `data/training_data`.

In [2]:
from datasets import load_dataset
import os

# Ścieżka do danych
current_dir = os.getcwd()
# Wychodzimy 2 poziomy w górę do głównego folderu
project_root = os.path.abspath(os.path.join(current_dir, "../../"))
data_file = os.path.join(project_root, "data", "training_data", "financial_sentiment_train.jsonl")

print(f"📂 Ładowanie danych z: {data_file}")

if not os.path.exists(data_file):
    raise FileNotFoundError("❌ Nie znaleziono pliku danych! Uruchom najpierw notatnik z Dnia 1.")

# Ładujemy dataset
dataset = load_dataset("json", data_files=data_file, split="train")

print(f"✅ Załadowano {len(dataset)} przykładów.")
print("Przykładowy rekord:", dataset[0])

📂 Ładowanie danych z: c:\Users\takze\OneDrive\Pulpit\project\financial-ai-engineering\data\training_data\financial_sentiment_train.jsonl
✅ Załadowano 8588 przykładów.
Przykładowy rekord: {'instruction': 'Analyze the sentiment of the following financial text. Answer with: Negative, Neutral, or Positive.', 'input': "One of the top-performing beauty stocks could surge after 'Kylie Jenner' makeover - CNBC", 'output': 'Neutral'}


### 📝 4. Formatowanie Promptu

Model uczy się na tekście. Musimy skleić nasze dane (instruction, input, output) w jeden ciąg znaków.
Użyjemy formatu **Alpaca**, który jest standardem dla małych modeli.

In [3]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    
    # Iterujemy po paczce danych
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Wzór promptu (musi być taki sam podczas treningu i używania!)
        text = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}""" # <-- Tutaj kończy się przykład treningowy
        
        texts.append(text)
    return { "text": texts }

print("✅ Funkcja formatująca gotowa.")

✅ Funkcja formatująca gotowa.


### ⚙️ 5. Konfiguracja QLoRA (Model 4-bit)

To najważniejszy techniczny moment.
1.  **BitsAndBytes:** Konfiguracja ładowania w 4-bitach (oszczędność pamięci VRAM).
2.  **LoRA:** Konfiguracja adapterów (małe warstwy, które będziemy trenować).
3.  **Model:** Użyjemy `Qwen/Qwen2.5-1.5B-Instruct` (jest lekki, szybki i bardzo dobry).

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig

# Wybór modelu
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"

print(f"⏳ Ładowanie modelu: {MODEL_ID}...")

# 1. Konfiguracja Kwantyzacji (4-bit) - POPRAWIONA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,  # ✅ FLOAT32 - eliminuje konflikty
    bnb_4bit_use_double_quant=False,
)

# 2. Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

# 3. Model (Baza) - bez torch_dtype
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    # ❌ Usuń torch_dtype - quantization_config to obsłuży
)

# 4. Konfiguracja LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

print("✅ Model załadowany na GPU w 4-bitach (float32 compute).")

⏳ Ładowanie modelu: Qwen/Qwen2.5-1.5B-Instruct...
✅ Model załadowany na GPU w 4-bitach (float32 compute).


### 🏋️‍♂️ 6. Uruchomienie Treningu (SFTTrainer)

Używamy `SFTTrainer` (Supervised Fine-Tuning Trainer).
To narzędzie, które bierze model, dane i konfigurację LoRA, a następnie "mieli" to na karcie graficznej.

Wynik zapiszemy w `data/models/finance-sentiment-lora`.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
import os

# Gdzie zapisać wynik?
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "../../"))
output_dir = os.path.join(project_root, "data", "models", "finance-sentiment-lora")

# Formatowanie datasetu
def format_dataset(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
        texts.append(text)
    
    return {"text": texts}

# Aplikuj formatowanie
formatted_dataset = dataset.map(
    format_dataset,
    batched=True,
    remove_columns=dataset.column_names
)

print(f"✅ Dataset przeformatowany. Liczba przykładów: {len(formatted_dataset)}")

# Wyłączenie TF32 dla precyzji
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

# Konfiguracja treningu
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=5,
    fp16=False,
    bf16=False,
    tf32=False,
    dataloader_pin_memory=False,
    optim="adamw_torch",
    save_strategy="no",
    report_to="none",
    use_cpu=False,
)

# Inicjalizacja trenera
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset,
    peft_config=peft_config,
)

print("🚀 ROZPOCZYNAM TRENING (z wyłączonym autocast)...")

# --- POPRAWKA TUTAJ ---
# Używamy nowej składni torch.amp.autocast zamiast torch.cuda.amp.autocast
with torch.amp.autocast('cuda', enabled=False):
    trainer.train()
# ----------------------

print("-" * 30)
print("💾 Zapisywanie adaptera...")
trainer.save_model(output_dir)
print(f"✅ Model zapisany w: {output_dir}")

### 🧪 7. Szybki Test (Inference)

Sprawdźmy "na gorąco", czy model działa.
Wygenerujemy odpowiedź dla przykładowego newsa.

In [6]:
# Testowy news
input_text = "The company reported a record loss due to supply chain issues."

# Formatujemy tak samo jak w treningu (tylko bez odpowiedzi)
prompt = f"""### Instruction:
Analyze the sentiment of the following financial news headline. Answer with: Negative, Neutral, or Positive.

### Input:
{input_text}

### Response:
"""

# Tokenizacja i wysłanie na GPU
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generowanie
with torch.no_grad():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=10, 
        pad_token_id=tokenizer.eos_token_id
    )

# Odczytanie wyniku
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

### Instruction:
Analyze the sentiment of the following financial news headline. Answer with: Negative, Neutral, or Positive.

### Input:
The company reported a record loss due to supply chain issues.

### Response:
Negative
